In [1]:
from warnings import filterwarnings 
filterwarnings("ignore")

import pandas as pd
import numpy as np

In [2]:
realtor_data_df = pd.read_csv('NewYork-realtor-data.csv')
realtor_data_df["prev_sold_year"] = pd.to_datetime(realtor_data_df["prev_sold_date"], format='%Y-%m-%d').dt.year

In [3]:
for_sale_df = realtor_data_df[(realtor_data_df["prev_sold_year"] > 2015.0) &
                                     (realtor_data_df["state"].isin(["New York"])) &
                                     (realtor_data_df["status"] == "for_sale") 
                                    ]
for_sale_df.drop(columns=["prev_sold_year"],inplace=True)
for_sale_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8061 entries, 2 to 32595
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      8061 non-null   int64  
 1   status          8061 non-null   object 
 2   bed             7646 non-null   float64
 3   bath            7628 non-null   float64
 4   acre_lot        6701 non-null   float64
 5   city            8061 non-null   object 
 6   state           8061 non-null   object 
 7   zip_code        8061 non-null   float64
 8   house_size      4193 non-null   float64
 9   prev_sold_date  8061 non-null   object 
 10  price           8061 non-null   float64
dtypes: float64(6), int64(1), object(4)
memory usage: 755.7+ KB


In [4]:
zipcodes = for_sale_df["zip_code"]
len(zipcodes.unique())

532

In [5]:
# Because of its size this dataset is not saved in the code repo for this project.
# It can be downloaded and saved in the same path to make this Notebook run.
zip_lon_lat = pd.read_csv('../../../zip_lat_long.csv')
zip_lon_lat.head()

,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


In [6]:
find_zipcodes_df = zip_lon_lat[zip_lon_lat["ZIP"].isin(zipcodes.unique())]
find_zipcodes_df["zip_code_str"] = '0'+ find_zipcodes_df.ZIP.apply(int).apply(str)
find_zipcodes_df.head()

,ZIP,LAT,LNG,zip_code_str
1974,7001,40.582316,-74.271506,07001
1975,7002,40.662338,-74.110275,07002
1976,7003,40.809128,-74.187155,07003
1977,7004,40.882508,-74.304593,07004
1978,7005,40.932771,-74.417304,07005


In [7]:
uniquezip_df = pd.DataFrame(zipcodes.unique())
uniquezip_df.rename(columns={0: "ZIP"},inplace=True)
uniquezip_df["zip_code_str"] = '0'+ uniquezip_df.ZIP.apply(int).apply(str)
uniquezip_df.head()

,ZIP,zip_code_str
0,7001.0,07001
1,7002.0,07002
2,7003.0,07003
3,7004.0,07004
4,7005.0,07005


merged_df.to_csv('zip-lon-lat.csv')

uniquezip.sort_values(0).to_csv("zipcodes.csv")

In [8]:
merge_df = uniquezip_df.merge(find_zipcodes_df, how='outer', on='zip_code_str')

In [9]:
merge_df[merge_df.isna().any(axis=1)]

,ZIP_x,zip_code_str,ZIP_y,LAT,LNG
502,8218.0,08218,NaN,NaN,NaN
510,8245.0,08245,NaN,NaN,NaN
512,8239.0,08239,NaN,NaN,NaN


In [10]:
# "08218","39.1386785","-74.856442"

In [11]:
# "08239","39.1979584","-74.6577965"

In [12]:
# "08245","39.1778125","-74.8300693"

In [13]:
match_zipcodes = find_zipcodes_df.merge(for_sale_df[['zip_code',"city"]], how='outer', left_on='ZIP', right_on='zip_code')
match_zipcodes[match_zipcodes.isna().any(axis=1)]

,ZIP,LAT,LNG,zip_code_str,zip_code,city
8058,NaN,NaN,NaN,NaN,8218.0,Goshen
8059,NaN,NaN,NaN,NaN,8245.0,Dennisville
8060,NaN,NaN,NaN,NaN,8239.0,Strathmere


In [ ]:
match_zipcodes.dropna(subset=['latitude'], inplace=True)